In [2]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import mindspore
import datasets
from tqdm import tqdm
from mindnlp.transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np

from mindnlp.peft import AdaLoraConfig, PeftConfig, PeftModel, TaskType, get_peft_model
from mindspore.dataset import GeneratorDataset


os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = "cuda"
model_name_or_path = "facebook/bart-base"
tokenizer_name_or_path = "facebook/bart-base"

checkpoint_name = "financial_sentiment_analysis_lora_v1.pt"
text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 1e-3
num_epochs = 8
batch_size = 8


[WARNING] ME(873872:140383887885504,MainProcess):2024-04-25-20:51:53.627.27 [mindspore/run_check/_check_version.py:102] MindSpore version 2.3.0rc1 and cuda version 12.2.140 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.


/home/daiyuxin/miniconda3/envs/lyk_ms2.3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.612 seconds.
Prefix dict has been built successfully.


In [3]:
# creating model
peft_config = AdaLoraConfig(
    init_r=12,
    target_r=8,
    beta1=0.85,
    beta2=0.85,
    tinit=200,
    tfinal=1000,
    deltaT=10,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


trainable params: 2,434,176 || all params: 141,854,688 || trainable%: 1.715964438200308


In [6]:
# download dataset
!wget https://hf-mirror.com/datasets/financial_phrasebank/resolve/main/data/FinancialPhraseBank-v1.0.zip
!unzip FinancialPhraseBank-v1.0.zip

--2024-04-25 20:54:44--  https://hf-mirror.com/datasets/financial_phrasebank/resolve/main/data/FinancialPhraseBank-v1.0.zip
Resolving hf-mirror.com (hf-mirror.com)... 153.121.57.40, 160.16.199.204, 133.242.169.68
Connecting to hf-mirror.com (hf-mirror.com)|153.121.57.40|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf-mirror.com/datasets/financial_phrasebank/0e1a06c4900fdae46091d031068601e3773ba067c7cecb5b0da1dcba5ce989a6?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27FinancialPhraseBank-v1.0.zip%3B+filename%3D%22FinancialPhraseBank-v1.0.zip%22%3B&response-content-type=application%2Fzip&Expires=1714308541&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDMwODU0MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9maW5hbmNpYWxfcGhyYXNlYmFuay8wZTFhMDZjNDkwMGZkYWU0NjA5MWQwMzEwNjg2MDFlMzc3M2JhMDY3YzdjZWNiNWIwZGExZGNiYTVjZTk4OWE2P3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9z

In [3]:
def process_dataset(source,   batch_size=32, shuffle=False):

    column_names = ['input_ids', 'attention_mask','labels','text_labels']
    
    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    # transforms

    dataset = dataset.batch(batch_size)

    return dataset

class MSDataset:
    def __init__(self, filepath,tokenizer,max_length):
        self.path = filepath
        self.sentences = []
        self.labels = []
        self.text_labels = []
        self._load()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def _load(self):
        label_mapping = {
            "negative": 0,
            "neutral": 1,
            "positive": 2
        }
        with open(self.path, encoding="iso-8859-1") as f:
            for line in f:
                sentence, label_text = line.strip().split("@")
                self.sentences.append(sentence)
                self.labels.append(label_mapping[label_text])
                self.text_labels.append(label_text)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        text_labels = self.text_labels[index]
        model_inputs = self.tokenizer(sentence, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="np")
        labels = self.tokenizer(text_labels, max_length=3, padding="max_length", truncation=True, return_tensors="np")
        labels = labels["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels,self.text_labels[index]

    def __len__(self):
        return len(self.sentences)
    
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
dataset = process_dataset(MSDataset("./FinancialPhraseBank-v1.0/Sentences_AllAgree.txt",tokenizer,max_length),batch_size=batch_size)

train_dataset, eval_dataset = dataset.split([0.9, 0.1])



In [4]:
optimizer = mindspore.nn.AdamWeightDecay(model.trainable_params(), learning_rate=lr)


In [5]:
print(next(train_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[8, 1, 128], dtype=Int64, value=
[[[    0, 46583,    16 ...     1,     1,     1]],
 [[    0,   717,  5160 ...     1,     1,     1]],
 [[    0,  9682,   438 ...     1,     1,     1]],
 ...
 [[    0, 27602, 28477 ...     1,     1,     1]],
 [[    0,   717,  2492 ...     1,     1,     1]],
 [[    0,  9089,   523 ...     1,     1,     1]]]), 'attention_mask': Tensor(shape=[8, 1, 128], dtype=Int64, value=
[[[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]],
 ...
 [[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]]]), 'labels': Tensor(shape=[8, 1, 3], dtype=Int64, value=
[[[    0, 12516,     2]],
 [[    0, 12516,     2]],
 [[    0, 12516,     2]],
 ...
 [[    0, 12516,     2]],
 [[    0, 12516,     2]],
 [[    0, 12516,     2]]]), 'text_labels': Tensor(shape=[8], dtype=String, value= ['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
 'neutral', 'neutral'])}


In [6]:
model.base_model.peft_config["default"].total_step = len(train_dataset) * num_epochs
from mindspore import Tensor

num_batches = len(train_dataset)
num_batches_eval = len(eval_dataset)
                       
def forward_fn(input_ids,attention_mask,labels ):
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        return output.loss, output.logits
    
grad_fn = mindspore.value_and_grad(
        forward_fn, None,model.trainable_params(), has_aux=True,return_ids=True
    )
global_step = 0
for epoch in range(num_epochs):
    model.set_train(True)
    total_loss, total_step = 0, 0
    correct = 0
    total = 0
    with tqdm(total=num_batches) as t:
        for step, (input_ids,attention_mask,labels,_) in enumerate(train_dataset):
            input_ids  = input_ids.squeeze(axis=1)
            labels  = labels.squeeze(axis=1)
            attention_mask = attention_mask.squeeze(axis=1)
            (loss, logits), grad = grad_fn(input_ids,attention_mask,labels)
            gradient = [g for _, g in grad]
            gradient = tuple(gradient)
            optimizer(gradient)
            model.base_model.update_and_allocate(global_step, grad)
            total_loss += loss.asnumpy()
            total_step += 1
            global_step += 1
            curr_loss = total_loss / total_step
            t.set_postfix({'train-loss': f'{curr_loss:.2f}'})
            t.update(1)
    model.set_train(False)
    eval_loss = 0
    total_step = 0
    eval_preds = []
    text_labels = []
    with tqdm(total=num_batches_eval) as t:
        for step, (input_ids,attention_mask,labels,text) in enumerate(eval_dataset):
            input_ids = input_ids.squeeze(axis=1)
            labels = labels.squeeze(axis=1)
            attention_mask = attention_mask.squeeze(axis=1)
            outputs = model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
            loss = outputs.loss
            eval_loss += loss.asnumpy()
            total_step += 1           
            eval_loss = total_loss / total_step
            eval_preds.extend(
                tokenizer.batch_decode(np.argmax(outputs.logits.asnumpy(), -1), skip_special_tokens=True)
            )
            text_str = str(text.asnumpy())
            text_str = text_str.replace("[", "").replace("]", "").replace(",", "").replace("'", "")
            labels = text_str.split(" ")
            text_labels.extend(labels)
            t.set_postfix({'eval-loss': f'{eval_loss:.2f}'})
            t.update(1)
    for pred, text_label in zip(eval_preds, text_labels):
        if pred.strip() == text_label.strip():
            correct += 1
        total += 1
    accuracy = correct / total * 100
    print(f"{accuracy=} % on the evaluation dataset")
    eval_epoch_loss = eval_loss / eval_dataset.get_dataset_size()
    eval_ppl = np.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / train_dataset.get_dataset_size()
    train_ppl = np.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

  0%|          | 0/255 [00:00<?, ?it/s]

: 

In [6]:
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [7]:
ckpt = f"{peft_model_id}/adapter_model.ckpt"


In [8]:
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

[WARNING] ME(857257:140373268870336,MainProcess):2024-04-25-18:53:07.740.385 [mindspore/train/serialization.py:1456] For 'load_param_into_net', 355 parameters in the 'net' are not loaded, because they are not in the 'parameter_dict', please check whether the network structure is consistent when training and loading checkpoint.
[WARNING] ME(857257:140373268870336,MainProcess):2024-04-25-18:53:07.741.240 [mindspore/train/serialization.py:1460] ['base_model.model.shared.weight', 'base_model.model.encoder.embed_positions.weight', 'base_model.model.encoder.layers.0.self_attn.k_proj.weight', 'base_model.model.encoder.layers.0.self_attn.k_proj.bias', 'base_model.model.model.encoder.layers.0.self_attn.k_proj.ranknum.default', 'base_model.model.encoder.layers.0.self_attn.v_proj.weight', 'base_model.model.encoder.layers.0.self_attn.v_proj.bias', 'base_model.model.model.encoder.layers.0.self_attn.v_proj.ranknum.default', 'base_model.model.encoder.layers.0.self_attn.q_proj.weight', 'base_model.mod